In [174]:
import torch 
import torchvision
from torch.utils.data import Dataset
import cv2
import glob
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [147]:
data_path_1_po = './raw_data/validation_data/고양이/안구/일반/각막궤양/유'
data_path_1_ne = './raw_data/validation_data/고양이/안구/일반/각막궤양/무'

data_path_2_po = './raw_data/validation_data/고양이/안구/일반/각막부골편/유'
data_path_2_ne = './raw_data/validation_data/고양이/안구/일반/각막부골편/무'

data_path_3_po = './raw_data/validation_data/고양이/안구/일반/결막염/유'
data_path_3_ne = './raw_data/validation_data/고양이/안구/일반/결막염/무'

data_path_4_po = './raw_data/validation_data/고양이/안구/일반/비궤양성각막염/유'
data_path_4_ne = './raw_data/validation_data/고양이/안구/일반/비궤양성각막염/무'

data_path_5_po = './raw_data/validation_data/고양이/안구/일반/안검염/유'
data_path_5_ne = './raw_data/validation_data/고양이/안구/일반/안검염/무'

paths = [data_path_1_po, data_path_1_ne, data_path_2_po, data_path_2_ne, data_path_3_po, data_path_3_ne, data_path_4_po,
         data_path_4_ne, data_path_5_po, data_path_5_ne ]


In [153]:
image_path = []
labels = []

def get_data_path(data_path):
    for path in glob.glob(data_path+'/*'): 
        if path[-3:]=="jpg":
            image_path.append(path)
            if data_path[-1:] == "무":
                labels.append("무")
            else:
                labels.append(data_path.split('/')[6])

for path in paths: 
    get_data_path(path)

print(len(labels))
print(len(image_path))

2918
2918


In [168]:
label_mappings = {"무":0,"각막부골편":1,"안검염":2,"비궤양성각막염":3,"각막궤양":4,"결막염":5}

def label_encoder(labels,mappings):
    encoded_labels = []
    for label in labels:
        temp = mappings[label]
        encoded_labels.append(temp)

    return encoded_labels

encoded_labels = label_encoder(labels, label_mappings)
print(encoded_labels[:5])
print(len(encoded_labels))

[4, 4, 4, 4, 4]
2918


In [177]:
train_image_path, test_image_path, train_label, test_label = train_test_split(image_path, encoded_labels, test_size=0.2,shuffle=True,stratify=encoded_labels )
train_image_path, val_image_path, train_label, val_label = train_test_split(train_image_path,train_label,test_size=0.1,shuffle=True,stratify=train_label)

print(len(train_image_path))
print(len(train_label))
print(len(val_image_path))
print(len(val_label))
print(len(test_image_path))
print(len(test_label))

2100
2100
234
234
584
584


In [196]:
class CustomDataset(Dataset):
    def __init__(self, image_paths,labels,transform=False): 
        self.image_paths = image_paths
        self.labels = labels 
        self.transform = transform
    def __len__(self):
        return len(self.image_paths) 
    
    def __getitem__(self,idx):
        image_filepath = self.image_paths[idx]
        image = cv2.imread(image_filepath)

        label = self.labels

        if self.transform is not None: 
            image = self.transform(image)

        return image, label 